In [17]:
import subprocess
from concurrent.futures import ProcessPoolExecutor

import numpy as np

In [2]:
def assembly_accuracy(reference, assembly, output):
    program = '/media/GenomicResearch/Tools/assembly_accuracy/fastmer.py'
    command = [program, '--reference', reference, '--assembly', assembly, '--min-mapping-quality', '10']
    with open(output, 'wb') as f:
        subprocess.run(
            command,
            stdout=f
        )

In [21]:
def quality_score(reference, query):
    p = subprocess.Popen(['minimap2', '-c', reference, query], stdout=subprocess.PIPE)
    output = subprocess.check_output(
        ['perl', '-ane', 'if(/tp:A:P/&&/NM:i:(\d+)/){$n+=$1;$m+=$1 while/(\d+)M/g;$g+=$1,++$o while/(\d+)[ID]/g}END{print(($n-$g+$o)/($m+$o),"\n")}'],
        stdin=p.stdout
    )
    return -10 * np.log10(float(output.decode()))

In [3]:
from pathlib import Path

In [48]:
dirpath = Path('/media/GenomicResearch/Issue/20210909_estimate_nanopore_depth/wgs-2/20211209_Lis_barcode10_R20-0148/denovo')
outpath = Path('/media/GenomicResearch/Issue/20210909_estimate_nanopore_depth/wgs-2/20211209_Lis_barcode10_R20-0148/assembly_accuracy/')

In [49]:
with ProcessPoolExecutor(64) as executor:
    for i in dirpath.iterdir():
        reference = '/media/GenomicResearch/Issue/20210909_estimate_nanopore_depth/wgs-2/20211209_Lis_barcode10_R20-0148/reference.fa'
        output = outpath/(i.name + '.txt')
        executor.submit(assembly_accuracy, reference, i/'contigs.fasta', output)

In [51]:
dirpath_1 = Path('/media/GenomicResearch/Issue/20220215_modification_paper_data/ONT_WGS/Contigs')
dirpath_2 = Path('/media/GenomicResearch/Issue/20220215_modification_paper_data/RefSeqs/Contigs')
outpath = Path('/media/GenomicResearch/Issue/20220215_modification_paper_data/ONT_WGS/Assembly_accuracy')

In [52]:
for query in dirpath_1.iterdir():
    reference = dirpath_2/query.name
    output = outpath/(query.stem + '.txt')
    assembly_accuracy(reference, query, output)